In [1]:
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.models import Sequential, Model
from tensorflow.python.keras.layers import Dropout, Flatten, Dense, Input
from tensorflow.python.keras import applications
from tensorflow.python.keras.layers import AveragePooling2D, GlobalAveragePooling2D
from tensorflow.python.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, History, TensorBoard
from tensorflow.python.keras.applications.mobilenet import MobileNet
from tensorflow.python.keras.layers import Conv2D, Convolution2D, MaxPooling2D, ZeroPadding2D, BatchNormalization, Activation
from tensorflow.python.keras.optimizers import RMSprop
from tensorflow.python.keras import backend as K
import numpy as np
import pandas as pd
from tensorflow.python.keras import layers
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tensorflow.python.keras.models import load_model

In [2]:
#parameters
classes = 12
batch_size=32
train_total = 3820
validation_total = 930

In [3]:
model_mobilenet = MobileNet(include_top=False, weights = 'imagenet',input_shape=(224,224,3), alpha=0.5)

In [4]:
model_mobilenet.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 16)      432       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 16)      64        
_________________________________________________________________
conv1_relu (Activation)      (None, 112, 112, 16)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 16)      144       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 16)      64        
_________________________________________________________________
conv_dw_1_relu (Activation)  (None, 112, 112, 16)      0         
__________

In [5]:
x = model_mobilenet.output
x = Dropout(0.5)(x)
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(classes, activation='softmax')(x)

In [6]:
model = Model(inputs=model_mobilenet.input, outputs=x)

In [7]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 16)      432       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 16)      64        
_________________________________________________________________
conv1_relu (Activation)      (None, 112, 112, 16)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 16)      144       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 16)      64        
_________________________________________________________________
conv_dw_1_relu (Activation)  (None, 112, 112, 16)      0         
__________

In [8]:
print(model.layers[0].trainable)

True


In [9]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range = 0.2,
                                   rotation_range = 20,
                                   height_shift_range=0.2,
                                   width_shift_range=0.2,
                                   zoom_range=0.2,
                                   fill_mode='reflect',
                                   horizontal_flip=True,
                                   vertical_flip=True)
validation_datagen = ImageDataGenerator(rescale = 1./255)

In [10]:
train_generator = train_datagen.flow_from_directory(
                    'train',
                    target_size=(224,224),
                    batch_size=batch_size,
                    class_mode='categorical',
                    shuffle=True
                    )
validation_generator = validation_datagen.flow_from_directory(
                        'validation',
                        target_size=(224,224),
                        batch_size=batch_size,
                        class_mode='categorical')

Found 3820 images belonging to 12 classes.
Found 930 images belonging to 12 classes.


In [11]:
def accuracy_temp(y_true, y_pred):
    
    return K.mean(K.equal(K.argmax(y_true),K.argmax(y_pred,axis=-1)))
    
        

In [54]:
model.compile(loss='categorical_crossentropy',
             optimizer=RMSprop(lr=1e-6),
             metrics=['accuracy'])

In [55]:
checkpointer = ModelCheckpoint(filepath='mobilenet_from_scratch_save_best_final_2.hdf5', verbose=1, save_best_only=True, save_weights_only=True)
reduceLR = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, min_lr=0)
earlystopping = EarlyStopping(monitor='val_loss', patience=20)
history_call = History()
tensorboard = TensorBoard(log_dir='./logs', write_graph=True)

In [56]:
history = model.fit_generator(
                    train_generator,
                    steps_per_epoch = int(np.ceil(train_total/batch_size)),
                    epochs=100,
                    validation_data=validation_generator,
                    validation_steps= int(np.ceil(validation_total/batch_size)),
                    verbose=2,
                    callbacks=[checkpointer, reduceLR, earlystopping, history_call, tensorboard])

Epoch 1/100
Epoch 00000: val_loss improved from inf to 0.12327, saving model to mobilenet_from_scratch_save_best_final_2.hdf5
76s - loss: 0.0998 - acc: 0.9660 - val_loss: 0.1233 - val_acc: 0.9624
Epoch 2/100
Epoch 00001: val_loss did not improve
69s - loss: 0.0930 - acc: 0.9673 - val_loss: 0.1233 - val_acc: 0.9624
Epoch 3/100
Epoch 00002: val_loss improved from 0.12327 to 0.12312, saving model to mobilenet_from_scratch_save_best_final_2.hdf5
69s - loss: 0.0834 - acc: 0.9687 - val_loss: 0.1231 - val_acc: 0.9634
Epoch 4/100
Epoch 00003: val_loss did not improve
71s - loss: 0.0846 - acc: 0.9684 - val_loss: 0.1234 - val_acc: 0.9624
Epoch 5/100
Epoch 00004: val_loss did not improve
71s - loss: 0.0892 - acc: 0.9701 - val_loss: 0.1236 - val_acc: 0.9624
Epoch 6/100
Epoch 00005: val_loss did not improve
69s - loss: 0.0882 - acc: 0.9709 - val_loss: 0.1237 - val_acc: 0.9634
Epoch 7/100
Epoch 00006: val_loss did not improve
71s - loss: 0.0815 - acc: 0.9707 - val_loss: 0.1243 - val_acc: 0.9624
Epoc

KeyboardInterrupt: 

In [57]:
model.load_weights('mobilenet_from_scratch_save_best_final_2.hdf5')

In [58]:
model.evaluate_generator(validation_generator, int(np.ceil(validation_total/batch_size)))

[0.12311724319374222, 0.96344086021505382]

In [59]:
validation_generator = validation_datagen.flow_from_directory(
                        'validation',
                        target_size=(224,224),
                        batch_size=batch_size,
                        class_mode='categorical',
                        shuffle=False)

Found 930 images belonging to 12 classes.


In [60]:
predictions_valid = model.predict_generator(validation_generator,int(np.ceil(validation_total/batch_size)))

In [61]:
predictions_valid = np.argmax(predictions_valid, axis=1)


In [62]:
true_positive = np.zeros(12)
false_positive = np.zeros(12)
true_negative = np.zeros(12)
false_negative = np.zeros(12)
for i in range(len(predictions_valid)):
    if(predictions_valid[i]==validation_generator.classes[i]):
        true_positive[predictions_valid[i]] = true_positive[predictions_valid[i]] + 1
    else:
        false_positive[predictions_valid[i]] = false_positive[predictions_valid[i]] + 1
        false_negative[validation_generator.classes[i]] = false_negative[validation_generator.classes[i]] +1

In [63]:
print('true_positive')
print(true_positive)
print('false_positive')
print(false_positive)
print('false_negative')
print(false_negative)
print('Total')
print(true_positive+false_negative)

true_positive
[  34.   94.   64.  107.   45.   86.  119.   36.  106.   40.   95.   70.]
false_positive
[ 12.   0.   0.   3.   4.   1.  11.   0.   1.   1.   0.   1.]
false_negative
[ 13.   0.   1.   1.   0.   3.  12.   0.   1.   2.   0.   1.]
Total
[  47.   94.   65.  108.   45.   89.  131.   36.  107.   42.   95.   71.]


In [68]:
predictions_valid

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  6,  0,  0,  0,  6,  0,  6,
        6,  0,  0,  5,  0,  0,  4,  6,  6,  0,  0,  0,  0,  4,  0,  0,  0,
        0,  0,  6,  0,  4,  0,  4,  0,  0,  0,  0,  6,  0,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2, 11,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,
        3,  3,  3,  3,  3

In [64]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_generator = test_datagen.flow_from_directory(
                    'test',
                    shuffle=False,
                    target_size=(224,224),
                    batch_size=batch_size,
                    class_mode=None)


Found 794 images belonging to 1 classes.


In [65]:
predictions = model.predict_generator(test_generator,int(np.ceil(794/batch_size)))
class_ids = {train_generator.class_indices[x]: x for x in train_generator.class_indices}
predicted_classes = [class_ids[x] for x in np.argmax(predictions, axis=1)]


In [66]:
from os.path import basename
test_ids = [basename(x) for x in test_generator.filenames]
submission = pd.DataFrame({'file':test_ids,'species':predicted_classes})
submission.to_csv('submission_mobilenet_from_scratch.csv', encoding="utf8", index=False)

In [67]:
from IPython.display import FileLink
FileLink('submission_mobilenet_from_scratch.csv')

/home/janardhan/Documents/plant_seeding/submission_mobilenet_from_scratch.csv

In [ ]:
# PB LB: 0.96977